In [6]:
from sklearn.datasets import  load_iris, fetch_california_housing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
from caas_jupyter_tools import display_dataframe_to_user
np.random.seed(0)

In [8]:
# dataset regression: california housing dataset
data = fetch_california_housing(as_frame=True)
# selectionner les 100 1eres lignes et 2 colonnes
X1 = data.data.loc[0:99, ['MedInc', 'HouseAge']]
# selectionner la cible correspondante
y1 = data.target.loc[0:99]
# afficher
print(X1.head())
print(y1.head())

/home/ange/.local/lib/python3.10/site-packages/sklearn/datasets/_base.py:1519: UserWarning: Retry downloading from url: https://ndownloader.figshare.com/files/5976036
  warnings.warn(f"Retry downloading from url: {remote.url}")


URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

In [7]:
# dataset classification: iris dataset
data = load_iris(as_frame=True)
df = data.frame
X2 = df[0:100][['petal width (cm)', 'petal length (cm)']]
y2 = df[0:100]['target']

print(X2.head())

   petal width (cm)  petal length (cm)
0               0.2                1.4
1               0.2                1.4
2               0.2                1.3
3               0.2                1.5
4               0.2                1.4


In [ ]:
# --- 1) Create small datasets ---
# Regression dataset (n < 100, 2 features)
n_reg = 50
X_reg = np.random.normal(size=(n_reg,2))
true_w_reg = np.array([3.0, -2.0])
y_reg = X_reg.dot(true_w_reg) + np.random.normal(scale=0.8, size=n_reg)
df_reg = pd.DataFrame(X_reg, columns=['x1','x2'])
df_reg['y'] = y_reg
display_dataframe_to_user("Jeu_de_donnees_regression", df_reg.head(50))

# Classification dataset (n < 100, 2 features, binary labels)
n_clf = 80
X_clf = np.random.normal(size=(n_clf,2))
true_w_clf = np.array([2.0, -1.0])
logit = X_clf.dot(true_w_clf)
prob = 1/(1+np.exp(-logit))
y_clf = (prob > 0.5).astype(float)
df_clf = pd.DataFrame(X_clf, columns=['x1','x2'])
df_clf['y'] = y_clf
display_dataframe_to_user("Jeu_de_donnees_classification", df_clf.head(80))

In [ ]:
# --- 2) Symbolic derivation for general MSE with 2 weights (no intercept) ---
w1, w2 = sp.symbols('w1 w2', real=True)
w = sp.Matrix([w1, w2])

In [ ]:
# Compute numeric XtX and Xty for regression dataset
X = sp.Matrix(X_reg)
y = sp.Matrix(y_reg)
n = X.rows
XtX = (X.T * X)  # 2x2
Xty = (X.T * y)  # 2x1

In [ ]:
# Build quadratic MSE expression: (1/n) * (w^T XtX w - 2 * (X^T y)^T w + y^T y)
const_term = (y.T * y)[0]
mse_expr = (1/sp.Integer(n))*( (w.T * XtX * w)[0] - 2*(Xty.T * w)[0] + const_term )
mse_expr_simpl = sp.simplify(mse_expr)

In [ ]:

# Gradient and Hessian symbolically (for our numerical XtX and Xty values but symbolic w1,w2)
grad = sp.Matrix([sp.diff(mse_expr_simpl, v) for v in (w1, w2)])
hess = sp.hessian(mse_expr_simpl, (w1, w2))

In [ ]:
# Compute analytic expressions in matrix form
# For general X: grad = (2/n) * X^T (X w - y) ; Hessian = (2/n) X^T X
grad_matrix_form = sp.Matrix(sp.factor((2/sp.Integer(n)) * (XtX * w - Xty)))
hess_matrix_form = sp.Matrix(sp.factor((2/sp.Integer(n)) * XtX))

In [ ]:
# Evaluate eigenvalues of Hessian (numeric) to check convexity
hess_num = np.array(hess_matrix_form.tolist(), dtype=float)
eigvals = np.linalg.eigvalsh(hess_num)

In [ ]:
# --- 3) Plotting loss surface (contours) for regression ---
# create grid for w1,w2
w1_vals = np.linspace(-1,5,200)
w2_vals = np.linspace(-5,3,200)
W1, W2 = np.meshgrid(w1_vals, w2_vals)
def mse_numeric(X_np, y_np, w1v, w2v):
    wv = np.vstack([w1v.ravel(), w2v.ravel()])
    preds = (X_np @ wv)
    # residual per column
    residuals = preds - y_np.reshape(-1,1)
    mse_vals = (residuals**2).mean(axis=0)
    return mse_vals.reshape(w1v.shape)

Z_reg = mse_numeric(X_reg, y_reg, W1, W2)

In [ ]:
# pick a point for tangent (user didn't specify) -> choose w0 = [1, 1]
w0 = np.array([1.0, 1.0])
mse_w0 = mse_numeric(X_reg, y_reg, np.array([[w0[0]]]), np.array([[w0[1]]]))[0,0]

In [ ]:
# compute gradient at w0 (numeric)
grad_func = lambda X_np, y_np, w: (2/X_np.shape[0]) * X_np.T.dot(X_np.dot(w) - y_np)
grad_w0 = grad_func(X_reg, y_reg, w0)

In [ ]:
# Tangent line: grad(MSE)(w0) . (w - w0) = 0 -> define line equation in w1,w2
a, b = grad_w0[0], grad_w0[1]
# line: a*(w1 - w0_1) + b*(w2 - w0_2) = 0 -> solve for w2
# w2 = w0_2 - (a/b)*(w1 - w0_1)  if b != 0
if abs(b) > 1e-12:
    tangent_line = lambda w1x: w0[1] - (a/b)*(w1x - w0[0])
else:
    tangent_line = None

In [ ]:
# Plot contour and tangent for regression
plt.figure(figsize=(6,5))
CS = plt.contour(W1, W2, Z_reg, levels=25)
plt.clabel(CS, inline=1, fontsize=8)
plt.scatter([w0[0]],[w0[1]])
plt.title("Contours de la MSE (régression) et point w0")
plt.xlabel("w1")
plt.ylabel("w2")
# draw ellipse level MSE(w0)
CS2 = plt.contour(W1, W2, Z_reg, levels=[mse_w0])
if tangent_line is not None:
    w1_plot = np.linspace(w1_vals.min(), w1_vals.max(), 200)
    plt.plot(w1_plot, tangent_line(w1_plot))
plt.xlim(w1_vals.min(), w1_vals.max())
plt.ylim(w2_vals.min(), w2_vals.max())
plt.show()

In [ ]:
# --- 4) Classification: compute MSE surface treating y in {0,1} ---
Z_clf = mse_numeric(X_clf, y_clf, W1, W2)

w0_clf = np.array([0.5, -0.5])
mse_w0_clf = mse_numeric(X_clf, y_clf, np.array([[w0_clf[0]]]), np.array([[w0_clf[1]]]))[0,0]
grad_w0_clf = grad_func(X_clf, y_clf, w0_clf)
a2, b2 = grad_w0_clf[0], grad_w0_clf[1]
if abs(b2) > 1e-12:
    tangent_line_clf = lambda w1x: w0_clf[1] - (a2/b2)*(w1x - w0_clf[0])
else:
    tangent_line_clf = None


In [ ]:
plt.figure(figsize=(6,5))
CS = plt.contour(W1, W2, Z_clf, levels=25)
plt.clabel(CS, inline=1, fontsize=8)
plt.scatter([w0_clf[0]],[w0_clf[1]])
plt.title("Contours de la MSE (classification, labels 0/1) et point w0_clf")
plt.xlabel("w1")
plt.ylabel("w2")
CS2 = plt.contour(W1, W2, Z_clf, levels=[mse_w0_clf])
if tangent_line_clf is not None:
    w1_plot = np.linspace(w1_vals.min(), w1_vals.max(), 200)
    plt.plot(w1_plot, tangent_line_clf(w1_plot))
plt.xlim(w1_vals.min(), w1_vals.max())
plt.ylim(w2_vals.min(), w2_vals.max())
plt.show()

In [ ]:
# --- 5) Prepare symbolic outputs to show to user ---
outputs = {
    "mse_expression_simplified": sp.simplify(mse_expr_simpl),
    "gradient_symbolic": grad,
    "hessian_symbolic": hess,
    "gradient_matrix_form": sp.simplify(grad_matrix_form),
    "hessian_matrix_form": sp.simplify(hess_matrix_form),
    "hessian_eigenvalues_numeric": eigvals,
    "grad_w0_regression": grad_w0,
    "mse_w0_regression": mse_w0,
    "tangent_line_regression_coeffs": (a, b, w0.tolist()),
    "grad_w0_classification": grad_w0_clf,
    "mse_w0_classification": mse_w0_clf,
    "tangent_line_classification_coeffs": (a2, b2, w0_clf.tolist())
}

outputs